#download data set

In [435]:
!pip install transformers

In [436]:
import pandas as pd
import numpy as np
import re

In [437]:
#setswana_url = 'https://raw.githubusercontent.com/NLPforLRLsProjects/SAfriSenti-Corpus/refs/heads/main/setswana_tweets.csv'
sesotho_tweets_url = 'https://raw.githubusercontent.com/Khotso-Bore/neural-languange-model/refs/heads/main/sesotho_tweets.csv'
sesotho_headlines_url = 'https://raw.githubusercontent.com/Khotso-Bore/neural-languange-model/refs/heads/main/Transformed_NewsSA_Dataset.csv'

#setswana_df = pd.read_csv(setswana_url)
sesotho_tweets_df = pd.read_csv(sesotho_tweets_url)
sesotho_headlines_df = pd.read_csv(sesotho_headlines_url)

In [438]:
def clean_text(text):
    # 1. Convert to lowercase
    text = text.lower()

    # 2. Remove numbers
    text = re.sub(r'\d+', '', text) # \d+ matches one or more digits

    # 3. Remove punctuation and special characters
    # This regex matches any character that is not a word character (alphanumeric + underscore) or whitespace.
    # It will also remove underscores if you don't want them. If you want to keep underscores,
    # use r'[^\w\s]' which means "not a word character AND not a whitespace character".
    text = re.sub(r'[^\w\s]', '', text) # Removes anything that isn't a letter, number, or underscore/whitespace.

    # 4. Remove extra whitespace (e.g., if punctuation/numbers were removed, leaving double spaces)
    text = re.sub(r'\s+', ' ', text).strip() # Replace multiple spaces with single space, then remove leading/trailing spaces

    return text

In [439]:
sesotho_tweets_df['cleaned_sentence'] = sesotho_tweets_df['sentence'].apply(clean_text).astype(str)
sesotho_headlines_df['cleaned_sentence'] = sesotho_headlines_df['sentence'].apply(clean_text).astype(str)

#tokenize dataset

In [440]:
#setswana_df['Final_Label'].unique()

In [441]:
sesotho_tweets_df.head()

,sentence,Final_labels,predict_name,cleaned_sentence
0,@user gwa tshwana rena ba bang a re kreye selo...,negative,Sesotho,user gwa tshwana rena ba bang a re kreye selo mos
1,e bata goal spurs,neutral,Sesotho,e bata goal spurs
2,@user @user ke nahana taba eno ea ho batla ho ...,negative,multi,user user ke nahana taba eno ea ho batla ho kh...
3,@user lotho hle empa fela ke ipotela,positive,Sesotho,user lotho hle empa fela ke ipotela
4,@user @user keu utloa hantle,positive,Sesotho,user user keu utloa hantle


In [442]:
sesotho_headlines_df.head()

,sentence,label,cleaned_sentence
0,BASUOE BA QOSUOE KA PELAELO EA HO BOLAEA MOSHE...,-1,basuoe ba qosuoe ka pelaelo ea ho bolaea moshe...
1,TSEBA MARENA A SEHLOOHO A NAHA,0,tseba marena a sehlooho a naha
2,LINTLHA-KHOLO MABAPI LE NTLO E OETSENG BATHO,-1,lintlhakholo mabapi le ntlo e oetseng batho
3,MOTHO O KHAOTSOE BOTONA LE MENOANA A NTSE A PHELA,-1,motho o khaotsoe botona le menoana a ntse a phela
4,LITABA TSE BOHLOKO HO MAQHEKU LE MAQHEKOANA NA...,-1,litaba tse bohloko ho maqheku le maqhekoana na...


In [443]:
sesotho_headlines_df['label'] = sesotho_headlines_df['label'].str.replace(r'[a-zA-Z]', '', regex=True)
sesotho_headlines_df['label'] = sesotho_headlines_df['label'].astype(int)
sesotho_headlines_df['label'] = np.where(sesotho_headlines_df['label'] == 0, 2, sesotho_headlines_df['label'])
sesotho_headlines_df['label'] = np.where(sesotho_headlines_df['label'] == -1, 0, sesotho_headlines_df['label'])
sesotho_headlines_df.head()

,sentence,label,cleaned_sentence
0,BASUOE BA QOSUOE KA PELAELO EA HO BOLAEA MOSHE...,0,basuoe ba qosuoe ka pelaelo ea ho bolaea moshe...
1,TSEBA MARENA A SEHLOOHO A NAHA,2,tseba marena a sehlooho a naha
2,LINTLHA-KHOLO MABAPI LE NTLO E OETSENG BATHO,0,lintlhakholo mabapi le ntlo e oetseng batho
3,MOTHO O KHAOTSOE BOTONA LE MENOANA A NTSE A PHELA,0,motho o khaotsoe botona le menoana a ntse a phela
4,LITABA TSE BOHLOKO HO MAQHEKU LE MAQHEKOANA NA...,0,litaba tse bohloko ho maqheku le maqhekoana na...


In [444]:
sesotho_headlines_df.rename(columns={'label': 'Final_Label'}, inplace=True)
sesotho_tweets_df.rename(columns={'Final_labels': 'Final_Label'}, inplace=True)

In [445]:
def label_to_int(x):
  if x == 'positive':
    return 1
  elif x == 'negative':
    return 0
  else:
    return 2

In [446]:
label_map = pd.DataFrame({
    'label': ['negative', 'positive', 'neutral'],
    'key': [0, 1, 2]
})

In [447]:
# value_counts = setswana_df['Final_Label'].value_counts().reset_index()
# value_counts.columns = ['label', 'count']
# value_counts_keys = pd.merge(label_map, value_counts, on='label', how='left').fillna(0)
# value_counts_keys['count'] = value_counts_keys['count'].astype(int)

In [448]:
#value_counts_keys

In [449]:
value_counts = sesotho_tweets_df['Final_Label'].value_counts().reset_index()
value_counts.columns = ['label', 'count']
value_counts_keys = pd.merge(label_map, value_counts, on='label', how='left').fillna(0)
value_counts_keys['count'] = value_counts_keys['count'].astype(int)

In [450]:
value_counts_keys

,label,key,count
0,negative,0,1464
1,positive,1,953
2,neutral,2,583


In [451]:
value_counts = sesotho_headlines_df['Final_Label'].value_counts().reset_index()
value_counts.columns = ['key', 'count']
value_counts_keys = pd.merge(label_map, value_counts, on='key', how='left').fillna(0)
value_counts_keys['count'] = value_counts_keys['count'].astype(int)
value_counts_keys

,label,key,count
0,negative,0,1552
1,positive,1,551
2,neutral,2,106


In [452]:
#setswana_df['Final_Label'] = [label_to_int(x) for x in setswana_df['Final_Label']]
sesotho_tweets_df['Final_Label'] = sesotho_tweets_df['Final_Label'].apply(label_to_int).astype(int)

In [453]:
value_counts = sesotho_tweets_df['Final_Label'].value_counts().reset_index()
value_counts.columns = ['key', 'count']
value_counts_keys = pd.merge(label_map, value_counts, on='key', how='left').fillna(0)
value_counts_keys['count'] = value_counts_keys['count'].astype(int)
value_counts_keys

,label,key,count
0,negative,0,1464
1,positive,1,953
2,neutral,2,583


In [454]:
sesotho_tweets_df.dtypes

,0
sentence,object
Final_Label,int64
predict_name,object
cleaned_sentence,object


In [455]:
value_counts = sesotho_headlines_df['Final_Label'].value_counts().reset_index()
value_counts.columns = ['key', 'count']
value_counts_keys = pd.merge(label_map, value_counts, on='key', how='left').fillna(0)
value_counts_keys['count'] = value_counts_keys['count'].astype(int)
value_counts_keys

,label,key,count
0,negative,0,1552
1,positive,1,551
2,neutral,2,106


In [456]:
sesotho_headlines_df.dtypes

,0
sentence,object
Final_Label,int64
cleaned_sentence,object


In [457]:
sesotho_df = pd.concat([sesotho_tweets_df, sesotho_headlines_df],axis=0,ignore_index=True)
sesotho_df['Final_Label'] = sesotho_df['Final_Label'].astype(int)

In [458]:
sesotho_df.columns

Index(['sentence', 'Final_Label', 'predict_name', 'cleaned_sentence'], dtype='object')

In [459]:
sesotho_df.shape

(5209, 4)

In [460]:
sesotho_df['Final_Label'].value_counts()

,count
Final_Label,
0,3016
1,1504
2,689


In [461]:
sesotho_df['cleaned_sentence'].shape

(5209,)

In [462]:
value_counts = sesotho_df['Final_Label'].value_counts().reset_index()
value_counts.columns = ['key', 'count']
value_counts_keys = pd.merge(label_map, value_counts, on='key', how='left').fillna(0)
value_counts_keys['count'] = value_counts_keys['count'].astype(int)
value_counts_keys

,label,key,count
0,negative,0,3016
1,positive,1,1504
2,neutral,2,689


In [463]:
sesotho_df.head()

,sentence,Final_Label,predict_name,cleaned_sentence
0,@user gwa tshwana rena ba bang a re kreye selo...,0,Sesotho,user gwa tshwana rena ba bang a re kreye selo mos
1,e bata goal spurs,2,Sesotho,e bata goal spurs
2,@user @user ke nahana taba eno ea ho batla ho ...,0,multi,user user ke nahana taba eno ea ho batla ho kh...
3,@user lotho hle empa fela ke ipotela,1,Sesotho,user lotho hle empa fela ke ipotela
4,@user @user keu utloa hantle,1,Sesotho,user user keu utloa hantle


#tokenize using subword tokenization

In [464]:
from transformers import XLMRobertaTokenizer, AutoTokenizer

In [465]:
bpe_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
word_piece_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [466]:
#setswana_df['tokens_bpe'] = setswana_df['sentence'].apply(lambda x: tokenizer.tokenize(x))
sesotho_df['bpe_tokens'] = sesotho_df['cleaned_sentence'].apply(lambda x: bpe_tokenizer.tokenize(x))
sesotho_df['word_piece_tokens'] = sesotho_df['cleaned_sentence'].apply(lambda x: word_piece_tokenizer.tokenize(x))

In [467]:
sesotho_df.head()

,sentence,Final_Label,predict_name,cleaned_sentence,bpe_tokens,word_piece_tokens
0,@user gwa tshwana rena ba bang a re kreye selo...,0,Sesotho,user gwa tshwana rena ba bang a re kreye selo mos,"[▁user, ▁gwa, ▁t, shwa, na, ▁rena, ▁ba, ▁bang,...","[user, g, ##wa, ts, ##hwa, ##na, ren, ##a, ba,..."
1,e bata goal spurs,2,Sesotho,e bata goal spurs,"[▁e, ▁bata, ▁goal, ▁, spur, s]","[e, bat, ##a, goal, spurs]"
2,@user @user ke nahana taba eno ea ho batla ho ...,0,multi,user user ke nahana taba eno ea ho batla ho kh...,"[▁user, ▁user, ▁ke, ▁na, hana, ▁taba, ▁en, o, ...","[user, user, ke, nah, ##ana, tab, ##a, en, ##o..."
3,@user lotho hle empa fela ke ipotela,1,Sesotho,user lotho hle empa fela ke ipotela,"[▁user, ▁lot, ho, ▁, hle, ▁emp, a, ▁fel, a, ▁k...","[user, lot, ##ho, h, ##le, em, ##pa, fe, ##la,..."
4,@user @user keu utloa hantle,1,Sesotho,user user keu utloa hantle,"[▁user, ▁user, ▁ke, u, ▁ut, lo, a, ▁han, tle]","[user, user, ke, ##u, ut, ##lo, ##a, han, ##tle]"


#process tokens

In [468]:
#setswana_encoding = tokenizer(setswana_df['sentence'].tolist(), padding=True, truncation=True, return_tensors='pt',max_length=64)
sesotho_bpe_encoding = bpe_tokenizer(sesotho_df['cleaned_sentence'].tolist(), padding=True, truncation=True, return_tensors='pt',max_length=64)
sesotho_word_piece_encoding = word_piece_tokenizer(sesotho_df['cleaned_sentence'].tolist(), padding=True, truncation=True, return_tensors='pt',max_length=64)

In [469]:
sesotho_bpe_encoding

{'input_ids': tensor([[    0, 38937, 39305,  ...,     1,     1,     1],
        [    0,    28,  8336,  ...,     1,     1,     1],
        [    0, 38937, 38937,  ...,   497,    10,     2],
        ...,
        [    0, 97549,    39,  ...,     1,     1,     1],
        [    0, 30078,  2590,  ...,     1,     1,     1],
        [    0, 22711,    28,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [470]:
from torch.utils.data import TensorDataset, DataLoader, random_split, Dataset
import torch
from sklearn.model_selection import train_test_split



In [471]:
def train_val_dataloader(encodings,data_labels):
  input_ids = encodings
  labels = data_labels

  dataset = TensorDataset(input_ids, labels)
  train_size = int(0.8 * len(dataset))
  val_size = len(dataset) - train_size

  train_ds, val_ds = random_split(dataset, [train_size, val_size])

  train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
  val_dl = DataLoader(val_ds, batch_size=32)

  return train_dl, val_dl

#Create neural model

In [472]:
import torch.nn as nn
import torch.nn.functional as F

class TweetClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(TweetClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.fc1 = nn.Linear(embed_dim * 64, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)              # (batch_size, seq_len, embed_dim)
        flat = embedded.view(x.size(0), -1)       # Flatten: (batch_size, seq_len * embed_dim)
        out = F.relu(self.fc1(flat))              # (batch_size, hidden_dim)
        return self.fc2(out)                      # (batch_size, output_dim)


In [473]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report


In [474]:
def train_eval_model(model, epochs,train_dl,val_dl):
    for epoch in range(epochs):
      model.train()
      total_loss = 0

      for xb, yb, in train_dl:
          preds = model(xb)
          loss = loss_fn(preds, yb)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          total_loss += loss.item()

      print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

    model.eval()
    correct, total = 0, 0

    predictions = []
    true_labels = []

    with torch.no_grad():
        for xb, yb in val_dl:
            preds = model(xb)
            predicted = torch.argmax(preds, dim=1)
            correct += (predicted == yb).sum().item()
            total += yb.size(0)

            predictions.extend(predicted)
            true_labels.extend(yb)


    print(f"Validation Accuracy: {correct / total:.2%}")

    return predictions, true_labels


In [475]:
def metrics(predictions, true_labels):

  # accuracy = accuracy_score(true_labels, predictions)

  # micro_precision = precision_score(true_labels, predictions, average='micro')
  # micro_recall = recall_score(true_labels, predictions, average='micro')
  # micro_f1 = f1_score(true_labels, predictions, average='micro')

  # print(f"Accuracy: {accuracy:.4%}")

  # print("\n--- Micro-averaged Metrics ---")
  # print(f"Micro-average Precision: {micro_precision:.4%}")
  # print(f"Micro-average Recall: {micro_recall:.4%}")
  # print(f"Micro-average F1-score: {micro_f1:.4%}")

  # macro_precision = precision_score(true_labels, predictions, average='macro', zero_division=0)
  # macro_recall = recall_score(true_labels, predictions, average='macro', zero_division=0)
  # macro_f1 = f1_score(true_labels, predictions, average='macro', zero_division=0)

  # print("\n--- Macro-averaged Metrics ---")
  # print(f"Macro-average Precision: {macro_precision:.4%}")
  # print(f"Macro-average Recall: {macro_recall:.4%}")
  # print(f"Macro-average F1-score: {macro_f1:.4%}")
  # print()

  # weighted_precision = precision_score(true_labels, predictions, average='weighted', zero_division=0)
  # weighted_recall = recall_score(true_labels, predictions, average='weighted', zero_division=0)
  # weighted_f1 = f1_score(true_labels, predictions, average='weighted', zero_division=0)

  # print("\n--- Weighted-averaged Metrics ---")
  # print(f"Weighted Precision: {weighted_precision:.4%}")
  # print(f"Weighted Recall:    {weighted_recall:.4%}")
  # print(f"Weighted F1-Score:  {weighted_f1:.4%}")

  class_names = ['negative','positive','neutral']
  print("\n--- Full Classification Report ---")
  print(classification_report(true_labels, predictions, target_names=class_names, zero_division=0))

## BPE model

In [476]:
model = TweetClassifier(vocab_size=bpe_tokenizer.vocab_size, embed_dim=64, hidden_dim=128, output_dim=3)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [477]:
train_dl, val_dl = train_val_dataloader(sesotho_bpe_encoding['input_ids'], torch.tensor(sesotho_df['Final_Label'].tolist()))

### evaluate model

In [478]:
predictions, true_labels = train_eval_model(model,10,train_dl,val_dl)

Epoch 1, Loss: 120.1564
Epoch 2, Loss: 103.2705
Epoch 3, Loss: 92.3839
Epoch 4, Loss: 80.4080
Epoch 5, Loss: 69.3226
Epoch 6, Loss: 56.6208
Epoch 7, Loss: 49.1436
Epoch 8, Loss: 39.1532
Epoch 9, Loss: 29.9598
Epoch 10, Loss: 24.5212
Validation Accuracy: 60.08%


In [479]:
metrics(np.array(predictions),np.array(true_labels))


--- Full Classification Report ---
              precision    recall  f1-score   support

    negative       0.67      0.80      0.73       595
    positive       0.48      0.37      0.42       297
     neutral       0.41      0.27      0.33       150

    accuracy                           0.60      1042
   macro avg       0.52      0.48      0.49      1042
weighted avg       0.58      0.60      0.58      1042



# STF-idf

In [480]:
from sklearn.feature_extraction.text import TfidfVectorizer
import torch

In [481]:
class STFIDFClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(STFIDFClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)


    def forward(self, x):
        out = F.relu(self.fc1(x))            # (batch_size, hidden_dim)
        return self.fc2(out)                      # (batch_size, output_dim)

In [482]:
sesotho_df

,sentence,Final_Label,predict_name,cleaned_sentence,bpe_tokens,word_piece_tokens
0,@user gwa tshwana rena ba bang a re kreye selo...,0,Sesotho,user gwa tshwana rena ba bang a re kreye selo mos,"[▁user, ▁gwa, ▁t, shwa, na, ▁rena, ▁ba, ▁bang,...","[user, g, ##wa, ts, ##hwa, ##na, ren, ##a, ba,..."
1,e bata goal spurs,2,Sesotho,e bata goal spurs,"[▁e, ▁bata, ▁goal, ▁, spur, s]","[e, bat, ##a, goal, spurs]"
2,@user @user ke nahana taba eno ea ho batla ho ...,0,multi,user user ke nahana taba eno ea ho batla ho kh...,"[▁user, ▁user, ▁ke, ▁na, hana, ▁taba, ▁en, o, ...","[user, user, ke, nah, ##ana, tab, ##a, en, ##o..."
3,@user lotho hle empa fela ke ipotela,1,Sesotho,user lotho hle empa fela ke ipotela,"[▁user, ▁lot, ho, ▁, hle, ▁emp, a, ▁fel, a, ▁k...","[user, lot, ##ho, h, ##le, em, ##pa, fe, ##la,..."
4,@user @user keu utloa hantle,1,Sesotho,user user keu utloa hantle,"[▁user, ▁user, ▁ke, u, ▁ut, lo, a, ▁han, tle]","[user, user, ke, ##u, ut, ##lo, ##a, han, ##tle]"
...,...,...,...,...,...,...
5204,KOMPONE E NEHELANA KA KOLOI HO SEPOLESA,1,NaN,kompone e nehelana ka koloi ho sepolesa,"[▁kompon, e, ▁e, ▁ne, hela, na, ▁ka, ▁kolo, i,...","[ko, ##mp, ##one, e, ne, ##hel, ##ana, ka, ko,..."
5205,MOSEBETSI OA HO NEHELANA LE HO BULA SEMMUSO MO...,1,NaN,mosebetsi oa ho nehelana le ho bula semmuso mo...,"[▁mos, e, bet, si, ▁o, a, ▁ho, ▁ne, hela, na, ...","[mo, ##se, ##bet, ##si, o, ##a, ho, ne, ##hel,..."
5206,MOHLOMPHEHI TONAKHOLO O FIHLA SETEREKENG MOO A...,1,NaN,mohlomphehi tonakholo o fihla seterekeng moo a...,"[▁mohlo, m, phe, hi, ▁tona, kho, lo, ▁o, ▁fi, ...","[mo, ##hl, ##omp, ##he, ##hi, ton, ##akh, ##ol..."
5207,KETEKELO EA KHOELI EA SEPOLESA,1,NaN,ketekelo ea khoeli ea sepolesa,"[▁kete, kel, o, ▁ea, ▁kho, eli, ▁ea, ▁se, pole...","[ke, ##tek, ##elo, ea, k, ##hoe, ##li, ea, sep..."


## N-gram

In [483]:

# Using character n-grams for TfidfVectorizer directly
stfidf_vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 4), min_df=2) # Example char 2-4-grams, min_df to filter rare ones
stf_idf_matrix = stfidf_vectorizer.fit_transform(sesotho_df['sentence'])
stf_idf_vocab_size = len(stfidf_vectorizer.get_feature_names_out())

# Convert to PyTorch tensor
stf_idf_tensor = torch.tensor(stf_idf_matrix.toarray(), dtype=torch.float32)
print(stf_idf_tensor.shape)
print(stf_idf_vocab_size)

torch.Size([5209, 22255])
22255


In [484]:
train_dl, val_dl = train_val_dataloader(stf_idf_tensor, torch.tensor(sesotho_df['Final_Label'].tolist()))

In [485]:
model = STFIDFClassifier(input_dim=stf_idf_vocab_size, hidden_dim=128, output_dim=3)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [486]:
predictions, true_labels = train_eval_model(model,10,train_dl,val_dl)

Epoch 1, Loss: 115.6522
Epoch 2, Loss: 82.4049
Epoch 3, Loss: 52.8748
Epoch 4, Loss: 31.3983
Epoch 5, Loss: 19.0172
Epoch 6, Loss: 11.6555
Epoch 7, Loss: 7.2094
Epoch 8, Loss: 4.7520
Epoch 9, Loss: 3.3428
Epoch 10, Loss: 2.6294
Validation Accuracy: 62.19%


In [487]:
metrics(np.array(predictions),np.array(true_labels))


--- Full Classification Report ---
              precision    recall  f1-score   support

    negative       0.71      0.78      0.74       632
    positive       0.45      0.38      0.41       285
     neutral       0.45      0.39      0.42       125

    accuracy                           0.62      1042
   macro avg       0.54      0.52      0.52      1042
weighted avg       0.61      0.62      0.61      1042



## BPE

In [488]:
subword_bpe_tokenised_documnets = [" ".join(s) for s in sesotho_df['bpe_tokens']]
subword_bpe_tokenised_documnets[0:3]

['▁user ▁gwa ▁t shwa na ▁rena ▁ba ▁bang ▁a ▁re ▁kre ye ▁se lo ▁mos',
 '▁e ▁bata ▁goal ▁ spur s',
 '▁user ▁user ▁ke ▁na hana ▁taba ▁en o ▁ea ▁ho ▁bat la ▁ho ▁khe tha ▁ho bane ▁re ▁she bile ▁our ▁own ▁benefits ▁re ▁le ▁bat ho ▁ke ▁e ona ▁e ▁sent seng ▁polo tik i ▁ea ▁les ot ho ▁so ▁we ▁went ▁behind ▁as sho les ▁for ▁years ▁co z ▁re ▁she bile ▁mele mo ▁e ▁direct ▁rele ▁bat ho ▁a ▁ee a ▁sebe let sa ▁n']

In [489]:

# Using character n-grams for TfidfVectorizer directly
stfidf_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=2)
stf_idf_matrix = stfidf_vectorizer.fit_transform(subword_bpe_tokenised_documnets)
stf_idf_vocab_size = len(stfidf_vectorizer.get_feature_names_out())


# Convert to PyTorch tensor
stf_idf_tensor = torch.tensor(stf_idf_matrix.toarray(), dtype=torch.float32)
print(stf_idf_tensor.shape)
print(stf_idf_vocab_size)


torch.Size([5209, 11834])
11834


In [490]:
train_dl, val_dl = train_val_dataloader(stf_idf_tensor, torch.tensor(sesotho_df['Final_Label'].tolist()))

In [491]:
model = STFIDFClassifier(input_dim=stf_idf_vocab_size, hidden_dim=64, output_dim=3)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [492]:
predictions, true_labels = train_eval_model(model,10,train_dl,val_dl)

Epoch 1, Loss: 125.0078
Epoch 2, Loss: 96.3057
Epoch 3, Loss: 69.0557
Epoch 4, Loss: 44.8863
Epoch 5, Loss: 28.2749
Epoch 6, Loss: 18.1956
Epoch 7, Loss: 12.2678
Epoch 8, Loss: 8.6947
Epoch 9, Loss: 6.6886
Epoch 10, Loss: 4.9169
Validation Accuracy: 62.09%


In [493]:
metrics(np.array(predictions),np.array(true_labels))


--- Full Classification Report ---
              precision    recall  f1-score   support

    negative       0.69      0.79      0.74       598
    positive       0.48      0.43      0.45       291
     neutral       0.53      0.31      0.40       153

    accuracy                           0.62      1042
   macro avg       0.57      0.51      0.53      1042
weighted avg       0.61      0.62      0.61      1042



## Word piece

In [494]:
subword_word_piece_tokenised_documnets = [" ".join(s) for s in sesotho_df['word_piece_tokens']]
subword_word_piece_tokenised_documnets[0:3]

['user g ##wa ts ##hwa ##na ren ##a ba bang a re k ##rey ##e se ##lo mo ##s',
 'e bat ##a goal spurs',
 'user user ke nah ##ana tab ##a en ##o ea ho bat ##la ho k ##het ##ha ho ##bane re she ##bil ##e our own benefits re le bath ##o ke e ##ona e sent ##sen ##g polo ##ti ##ki ea les ##otho so we went behind asshole ##s for years co ##z re she ##bil ##e mel ##em ##o e direct re ##le bath ##o a ee ##a se ##bel ##ets ##a n']

In [495]:
train_dl, val_dl = train_val_dataloader(stf_idf_tensor, torch.tensor(sesotho_df['Final_Label'].tolist()))

In [496]:
model = STFIDFClassifier(input_dim=stf_idf_vocab_size, hidden_dim=64, output_dim=3)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [497]:
predictions, true_labels = train_eval_model(model,10,train_dl,val_dl)

Epoch 1, Loss: 124.8548
Epoch 2, Loss: 96.6357
Epoch 3, Loss: 67.4265
Epoch 4, Loss: 42.6479
Epoch 5, Loss: 26.8098
Epoch 6, Loss: 17.1224
Epoch 7, Loss: 11.6707
Epoch 8, Loss: 8.0408
Epoch 9, Loss: 5.9385
Epoch 10, Loss: 4.6353
Validation Accuracy: 60.56%


In [498]:
metrics(np.array(predictions),np.array(true_labels))


--- Full Classification Report ---
              precision    recall  f1-score   support

    negative       0.68      0.78      0.73       607
    positive       0.45      0.40      0.42       302
     neutral       0.48      0.30      0.37       133

    accuracy                           0.61      1042
   macro avg       0.54      0.49      0.51      1042
weighted avg       0.59      0.61      0.59      1042



# Large Classifier

In [206]:
class LargeSTFIDFClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LargeSTFIDFClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, hidden_dim)
        self.fc5 = nn.Linear(hidden_dim, hidden_dim)
        self.fc6 = nn.Linear(hidden_dim, hidden_dim)
        self.fc7 = nn.Linear(hidden_dim, output_dim)


    def forward(self, x):
        out = F.relu(self.fc1(x))            # (batch_size, hidden_dim)
        out = F.relu(self.fc2(out))
        out = F.relu(self.fc3(out))
        out = F.relu(self.fc4(out))
        out = F.relu(self.fc5(out))
        out = F.relu(self.fc6(out))
        return self.fc7(out)

In [207]:
subword_bpe_tokenised_documnets = [" ".join(s) for s in sesotho_df['bpe_tokens']]

In [208]:
train_dl, val_dl = train_val_dataloader(stf_idf_tensor, torch.tensor(sesotho_df['Final_Label'].tolist()))
stfidf_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=2)
stf_idf_matrix = stfidf_vectorizer.fit_transform(subword_bpe_tokenised_documnets)
stf_idf_vocab_size = len(stfidf_vectorizer.get_feature_names_out())


# Convert to PyTorch tensor
stf_idf_tensor = torch.tensor(stf_idf_matrix.toarray(), dtype=torch.float32)

In [209]:
train_dl, val_dl = train_val_dataloader(stf_idf_tensor, torch.tensor(sesotho_df['Final_Label'].tolist()))

In [227]:
dims = [128,256]
epochs = [10,15,20]
lr = [1e-3, 1e-4, 3e-4]

for dim in dims:
  for epoch in epochs:
    for l_r in lr:

      print(f"\ndim: {dim}, epochs: {epoch}, lr: {l_r}")

      model = LargeSTFIDFClassifier(input_dim=stf_idf_vocab_size, hidden_dim=dim, output_dim=3)
      loss_fn = nn.CrossEntropyLoss()
      optimizer = torch.optim.Adam(model.parameters(), lr=l_r)

      predictions, true_labels = train_eval_model(model,epoch,train_dl,val_dl)
      metrics(np.array(predictions),np.array(true_labels))


dim: 128, epochs: 10, lr: 0.001
Epoch 1, Loss: 76.3735
Epoch 2, Loss: 56.5301
Epoch 3, Loss: 38.1398
Epoch 4, Loss: 29.2965
Epoch 5, Loss: 14.6838
Epoch 6, Loss: 5.9973
Epoch 7, Loss: 3.2209
Epoch 8, Loss: 2.0564
Epoch 9, Loss: 1.7951
Epoch 10, Loss: 1.3272
Validation Accuracy: 50.83%

--- Full Classification Report ---
              precision    recall  f1-score   support

    negative       0.61      0.66      0.63       297
    positive       0.39      0.42      0.41       184
     neutral       0.41      0.27      0.32       119

    accuracy                           0.51       600
   macro avg       0.47      0.45      0.45       600
weighted avg       0.50      0.51      0.50       600


dim: 128, epochs: 10, lr: 0.0001
Epoch 1, Loss: 81.7245
Epoch 2, Loss: 78.7084
Epoch 3, Loss: 75.3347
Epoch 4, Loss: 67.2081
Epoch 5, Loss: 55.0775
Epoch 6, Loss: 44.3703
Epoch 7, Loss: 33.9514
Epoch 8, Loss: 24.7374
Epoch 9, Loss: 16.7070
Epoch 10, Loss: 12.1801
Validation Accuracy: 51.50%

--

# Word tokenization

In [ ]:
# import nltk
# from nltk.tokenize import word_tokenize
# nltk.download('punkt_tab')



# tokenized = [word_tokenize(s.lower()) for s in sesotho_df['sentence'].tolist()]

# # Build a vocabulary
# vocab = {"<pad>": 0, "<unk>": 1}
# for sent in tokenized:
#     for word in sent:
#         if word not in vocab:
#             vocab[word] = len(vocab)


In [ ]:
# SEQ_LEN = 64

# def encode(sent):
#     tokens = word_tokenize(sent.lower())
#     token_ids = [vocab.get(tok, vocab["<unk>"]) for tok in tokens]
#     if len(token_ids) < SEQ_LEN:
#         token_ids += [vocab["<pad>"]] * (SEQ_LEN - len(token_ids))
#     else:
#         token_ids = token_ids[:SEQ_LEN]
#     return token_ids



In [ ]:
# class TweetDataset():
#     def __init__(self, texts, labels):
#         self.texts = texts
#         self.encoded_texts = torch.tensor([encode(t) for t in texts])
#         self.labels = labels

In [ ]:
# dataset = TweetDataset(setswana_df['sentence'].tolist(), setswana_df['Final_Label'].tolist())

In [ ]:
# dataset.texts[:5]

In [223]:
# dataset.encoded_texts[:5]

In [224]:
# model = TweetClassifier(vocab_size=len(vocab), embed_dim=64, hidden_dim=128, output_dim=3)
# loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [225]:
# train_dl, val_dl = train_val_dataloader(dataset.encoded_texts, torch.tensor(setswana_df['Final_Label'].tolist()))

In [226]:
# train_eval_model(model,10,train_dl,val_dl)